## Code Hist.

 - CODE : KIER Data_단순 분리 (전처리 X)  
 - DESC  
    &ensp; : int_Domain 코드의 변경에 따라,   
    &emsp; KIER 에너지 사용량 데이터를 동/층/세대별로 분리 (10분 주기)   
    &emsp; 추가적으로, Raw Data과 세대별 데이터 추출 단계에서, 아래와 같은 작업 진행  
    &emsp;&emsp; 1) Date Column에 대한 유효성 검사 및 이상 Data에 대한 소거  
    &emsp;&emsp; 1-1) Datetime 유효성 확인  
    &emsp;&emsp; 1-2) Datetime 이상치 제거  
    &emsp;&emsp; 2) HOUSE_ID_HO의 해시값을 int_house_num으로 변경 (용량 절감 목적)  
  - DATE  
    &ensp; 2023-10-12 Created  
    &ensp; 2023-11-21 Code 최신화  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 기존 모듈화된 함수 사용  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 기존 KIER 변환 관련 코드를 모두 통합  
    &ensp; 2024-01-19 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) HOT (온수 사용량) 전용 Code 작성  
    &ensp; 2024-03-13 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 적산 및 순시사용량 산출 코드 추가  
    &ensp; 2024-07-24 Error Fix : 호실별 사용량 취합시 MemoryError 발생 방지  
    &ensp; 2024-07-25 Code 개선 : HOUSE_ID Feature 도입에 따른 Code 변경  

## Code

### 01. Init

#### 01-01. Module Import

In [1]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import tqdm
from tqdm.notebook import tqdm
#endregion Import_Basic Module

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch, torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf, tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5382782383494674470
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22395486208
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8684608233463519986
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
 xla_global_id: 416903419]

In [4]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    print('Using GPU')
    tf.config.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)
else : 
  print('Using CPU')

Using GPU


In [5]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02

#### 01-02. Config (Directory, Params)

In [6]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [7]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-07-26 14:39:38.499635
2024 / 7 / 26
14 : 39


### 02. Data Prepare

In [8]:
## Dict_Domain
int_domain = 0

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)

## File
str_fileRaw, str_fileRaw_hList = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv'), str('KIER_hList_Common.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

0 : ELEC
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_ELEC_561-1-1_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-1_INST_01_10min.csv', 'KIER_ELEC_561-1-1_INST_10MIN.csv', 'KIER_ELEC_561-1-2_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-2_INST_01_10min.csv', 'KIER_ELEC_561-1-2_INST_10MIN.csv', 'KIER_ELEC_561-1-3_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-3_INST_01_10min.csv', 'KIER_ELEC_561-1-3_INST_10MIN.csv', 'KIER_ELEC_561-1-4_ACCU_01_Raw.csv', 'KIER_ELEC_561-1-4_INST_01_10min.csv', 

#### 02-01. KIER (hList)

In [9]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
# print(df_kier_hList.shape, ' /// ', df_kier_hList.columns)
# df_kier_hList

#### 02-02. Calculate Data Period

In [10]:
## 최적의 Period 계산
dt_start, dt_end = 0, 0

## 각 호별 Resampled Data 생성 및 취합
# df_kier_h_Combined = df_dt.copy()
cnt_h, df_kier_h_Combined = pd.DataFrame(), 0

list_HID = df_kier_hList['HOUSE_ID'].drop_duplicates()
for house in list_HID:
    print(str(house) + " H")

    str_file_h = str('KIER_' + str_domain + '_' + str(house) + '_ACCU_01_Raw.csv')
    df_h_tmp = pd.read_csv(str_dirName_h + str_file_h, index_col = 0)

    if dt_start == 0: 
        dt_start = df_h_tmp['METER_DATE'].min()
        print(dt_start)
    else : 
        if dt_start < df_h_tmp['METER_DATE'].min(): 
            print("True Start")
            dt_start = df_h_tmp['METER_DATE'].min()
    
    if dt_end == 0: 
        dt_end = df_h_tmp['METER_DATE'].max()
        print(dt_end)
    else : 
        if dt_start > df_h_tmp['METER_DATE'].max(): 
            print("True End")
            dt_start = df_h_tmp['METER_DATE'].max()

print('Calculated Period')
print(dt_start, ' / ', dt_end)

561-1-1 H
2022-07-17 23:20:00
2024-06-05 15:30:00
561-1-2 H
561-1-3 H
561-1-4 H
561-2-1 H
561-2-2 H
561-2-3 H
561-2-4 H
561-3-1 H
561-3-2 H
561-3-3 H
561-3-4 H
561-4-1 H
561-4-2 H
561-4-3 H
561-4-4 H
561-5-1 H
561-5-2 H
561-5-3 H
561-5-4 H
561-6-1 H
561-6-2 H
561-6-3 H
561-6-4 H
561-7-1 H
561-7-2 H
561-7-3 H
561-7-4 H
561-8-1 H
561-8-2 H
561-8-3 H
561-8-4 H
561-9-1 H
561-9-2 H
561-9-3 H
561-9-4 H
561-10-1 H
561-10-2 H
561-10-3 H
561-10-4 H
561-11-1 H
561-11-2 H
561-11-3 H
561-11-4 H
561-12-1 H
561-12-2 H
561-12-3 H
561-12-4 H
561-13-1 H
561-13-2 H
561-13-3 H
561-13-4 H
561-14-1 H
561-14-2 H
561-14-3 H
561-14-4 H
561-15-1 H
561-15-2 H
561-15-3 H
561-15-4 H
561-16-1 H
561-16-2 H
561-16-3 H
561-16-4 H
561-17-1 H
561-17-2 H
561-17-3 H
561-17-4 H
561-18-1 H
561-18-2 H
561-18-3 H
561-18-4 H
562-1-1 H
562-1-2 H
562-1-3 H
562-1-4 H
562-1-5 H
562-1-6 H
562-2-1 H
562-2-2 H
562-2-3 H
562-2-4 H
562-2-5 H
562-2-6 H
562-3-1 H
562-3-2 H
562-3-3 H
562-3-4 H
562-3-5 H
562-3-6 H
562-4-1 H
562-4-2 H
562-

In [11]:
## 여기서부터 실행하려 할 경우, Period를 지정해줘야함
## 계산된 Period에 대한 Date Dataframe 생성
df_kier_h_Combined = com_date.create_df_dt(pd.DataFrame(), 'METER_DATE', dt_start, dt_end, '10min')[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']]

## METER_DATE의 Minute이 10분 단위가 아닌 경우, 10분 단위로 변경
for i in range(0, len(df_kier_h_Combined)) : df_kier_h_Combined['MINUTE'].iloc[i] = (math.floor(df_kier_h_Combined['MINUTE'].iloc[i]/10))*10
df_kier_h_Combined

,YEAR,MONTH,DAY,HOUR,MINUTE
0,2022,7,17,23,20
1,2022,7,17,23,30
2,2022,7,17,23,40
3,2022,7,17,23,50
4,2022,7,18,0,0
...,...,...,...,...,...
99165,2024,6,5,14,50
99166,2024,6,5,15,0
99167,2024,6,5,15,10
99168,2024,6,5,15,20


### 03. 적산 사용량
모든 호실의 사용량을 변수화하여 한 데이터셋에 Combine

In [12]:
list_HID = df_kier_hList['HOUSE_ID'].drop_duplicates()
for house in list_HID:
    print(str(house) + " H")

    str_col_accu_h = str_col_accu + "_" + house
    str_file = 'KIER_' + str_domain + '_' + str(house) + '_ACCU_01_Raw.csv'
    df_h_tmp = pd.read_csv(str_dirName_h + str_file, index_col = 0).reset_index()[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu_h]]

    df_kier_h_Combined = pd.merge(df_kier_h_Combined, df_h_tmp, how = 'left', on = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'])
    print(df_kier_h_Combined.shape, ' /// ', df_kier_h_Combined.columns)

str_file = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
df_kier_h_Combined.to_csv(str_dirName_h + str_file)
print(df_kier_h_Combined.shape, ' /// ', df_kier_h_Combined.columns)
df_kier_h_Combined

561-1-1 H
(99170, 6)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'ELEC_ACTUAL_ACCU_EFF_561-1-1'],
      dtype='object')
561-1-2 H
(99170, 7)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'ELEC_ACTUAL_ACCU_EFF_561-1-1', 'ELEC_ACTUAL_ACCU_EFF_561-1-2'],
      dtype='object')
561-1-3 H
(99170, 8)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'ELEC_ACTUAL_ACCU_EFF_561-1-1', 'ELEC_ACTUAL_ACCU_EFF_561-1-2',
       'ELEC_ACTUAL_ACCU_EFF_561-1-3'],
      dtype='object')
561-1-4 H
(99170, 9)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'ELEC_ACTUAL_ACCU_EFF_561-1-1', 'ELEC_ACTUAL_ACCU_EFF_561-1-2',
       'ELEC_ACTUAL_ACCU_EFF_561-1-3', 'ELEC_ACTUAL_ACCU_EFF_561-1-4'],
      dtype='object')
561-2-1 H
(99170, 10)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'ELEC_ACTUAL_ACCU_EFF_561-1-1', 'ELEC_ACTUAL_ACCU_EFF_561-1-2',
       'ELEC_ACTUAL_ACCU_EFF_561-1-3', 'ELEC_ACTUAL_ACCU_EFF_561-1-4',
       'ELEC_ACTUAL_ACC

,YEAR,MONTH,DAY,HOUR,MINUTE,ELEC_ACTUAL_ACCU_EFF_561-1-1,ELEC_ACTUAL_ACCU_EFF_561-1-2,ELEC_ACTUAL_ACCU_EFF_561-1-3,ELEC_ACTUAL_ACCU_EFF_561-1-4,ELEC_ACTUAL_ACCU_EFF_561-2-1,...,ELEC_ACTUAL_ACCU_EFF_563-22-3,ELEC_ACTUAL_ACCU_EFF_563-22-4,ELEC_ACTUAL_ACCU_EFF_563-22-5,ELEC_ACTUAL_ACCU_EFF_563-22-6,ELEC_ACTUAL_ACCU_EFF_563-23-1,ELEC_ACTUAL_ACCU_EFF_563-23-2,ELEC_ACTUAL_ACCU_EFF_563-23-3,ELEC_ACTUAL_ACCU_EFF_563-23-4,ELEC_ACTUAL_ACCU_EFF_563-24-1,ELEC_ACTUAL_ACCU_EFF_563-24-2
0,2022,7,17,23,20,798.6900000000,650.6700000000,452.0000000000,812.2400000000,664.2700000000,...,524.5100000000,462.4000000000,489.8000000000,470.6400000000,358.2800000000,523.9700000000,441.8800000000,380.0800000000,393.9300000000,511.9200000000
1,2022,7,17,23,30,798.8600000000,650.8400000000,452.0400000000,812.5300000000,664.4500000000,...,524.6600000000,462.5100000000,489.8300000000,470.6800000000,358.3300000000,524.0500000000,441.9400000000,380.1400000000,393.9600000000,512.0200000000
2,2022,7,17,23,40,799.0200000000,651.0200000000,452.0700000000,812.7400000000,664.5800000000,...,524.7800000000,462.6100000000,489.8500000000,470.7200000000,358.3900000000,524.1100000000,441.9900000000,NaN,393.9800000000,512.1100000000
3,2022,7,17,23,50,799.1600000000,651.2500000000,452.1000000000,812.9800000000,664.7200000000,...,524.8800000000,462.7100000000,489.8900000000,470.7500000000,358.4400000000,524.1700000000,442.0400000000,380.2400000000,394.0100000000,512.2000000000
4,2022,7,18,0,0,799.3000000000,651.4600000000,452.1200000000,813.2500000000,664.8700000000,...,525.0000000000,462.8100000000,489.9300000000,470.7900000000,358.4800000000,524.2300000000,442.0800000000,380.2900000000,394.0400000000,512.2800000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99165,2024,6,5,14,50,10755.0000000000,9350.7800000000,7648.3000000000,10798.9000000000,11286.6500000000,...,NaN,NaN,7520.4100000000,NaN,6486.7300000000,NaN,NaN,NaN,NaN,NaN
99166,2024,6,5,15,0,10755.0400000000,9350.8400000000,7648.5300000000,10798.9300000000,11286.7500000000,...,NaN,NaN,7520.4500000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99167,2024,6,5,15,10,10755.0700000000,9350.9000000000,7648.8700000000,10798.9600000000,11286.8200000000,...,NaN,NaN,7520.5300000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99168,2024,6,5,15,20,10755.1200000000,9350.9700000000,7649.1600000000,10798.9800000000,11286.8900000000,...,NaN,6405.4600000000,7520.5800000000,NaN,6486.7800000000,NaN,6719.9600000000,NaN,NaN,7788.9200000000


In [13]:
# ## [미사용] Interpolation 후 수행해야할 연산
# list_col = df_kier_h_Combined.columns[5:] ## 세대별 에너지 사용량 Columns
# df_kier_Calc = df_kier_h_Combined[list_col]
# print(list_col)

# ## df_kier_h_Combined는 차후 "04. 순시사용량" 과정에서 재활용되므로, 
# ## df_kier_extract를 별도 선언 및 추출
# df_kier_extract = df_kier_h_Combined

# ## 사용량 평균 및 합계 Column 추가
# df_kier_extract['MEAN_OF_ACCU'], df_kier_extract['SUM_OF_ACCU'] = df_kier_Calc.mean(axis = 1), df_kier_Calc.sum(axis = 1)

# str_file = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
# df_kier_extract.to_csv(str_dirName_h + str_file)
# print(df_kier_extract.isnull().sum())
# df_kier_extract